In [1]:
import sys
if "../src/" not in sys.path:
    sys.path.append("../src/")
from roplot import FieldPowerPlay, Robot, StateRecord, DeclarativeBase
from datetime import datetime
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, desc
from sqlalchemy.orm import Session

from shapely import Point

In [9]:
fpp = FieldPowerPlay()

DATA_PATH='../src/data'
TODAY = datetime.now().strftime("%Y-%m-%d")
ENGINE = create_engine(f"sqlite:///{DATA_PATH}/{TODAY}.sqlite")

session = Session(ENGINE)
sr = session.query(StateRecord).order_by(desc( StateRecord.id )).limit(1).first()
sr.record

{'runtime': 32.399809,
 'x': 32.91096940489993,
 'y': 46.88261017312812,
 'heading': 323.8389099999999,
 'vx': 0.7760131293581211,
 'vy': 0.643576870936501,
 'heading_rate': 10}

In [3]:
from branca.element import Figure
import geopandas as gpd
import folium
crs = "Simple"
m2 = folium.Map(location=[72, 72], zoom_start=1, tiles=None, crs=crs)
fpp.add_to(m2, show_junctions=True)

# folium.GeoJson(gpd.GeoSeries(field).to_json()).add_to(m2)
m2.add_child(r, name="Robot")
# r.add_to(m2)

print(m2.__class__)
m2

NameError: name 'r' is not defined

In [ ]:
# With Robot as a GeoJson object, changing the robot position dynamically changes the resulting map
# without needing to re-generate the robot and put it in the map manually.
r.position = (86, 64) 
r.heading = 45
m2

# Potential solution: Make Robot a inherit from GeoJson and add it as an object directly. 
# Override the appropriate functions (hard part).

In [ ]:
ax = plt.gca()
fpp.draw_on_axes(ax)
r.draw_on_axes(ax)

In [ ]:
from folium import GeoJson
from typing import Callable

fig2 = Figure(width=550, height=550)
m1=folium.Map(location=[72, 72], zoom_start=1, tiles=None, crs=crs)

def style_line_color(color:str) -> Callable:
    def sf(nonsense):
        return {"color": color}
    return sf
    
for color in ("red", "blue"):
    gj = GeoJson(gpd.GeoSeries(fpp.field_lines.by_color(color)).to_json(),
                style_function=style_line_color(color))
    gj.add_to(m1)
r.add_to(m1)
# GeoJson().add_to(m2)
m1

In [ ]:
from datetime import datetime
datetime.now().strftime("%Y-%m-%d")

In [ ]:
r.position = (64, 64)
r.heading = -90
m1

In [ ]:
m2._children